# GPU Check

In [ ]:
!nvidia-smi

# GFPGAN Inference Demo 
### (No colorization; No CUDA extensions required)

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2101.04061)
[![GitHub Stars](https://img.shields.io/github/stars/TencentARC/GFPGAN?style=social)](https://github.com/TencentARC/GFPGAN)
[![download](https://img.shields.io/github/downloads/TencentARC/GFPGAN/total.svg)](https://github.com/TencentARC/GFPGAN/releases)

## GFPGAN - Towards Real-World Blind Face Restoration with Generative Facial Prior

GFPGAN is a blind face restoration algorithm towards real-world face images. <br>
It leverages the generative face prior in a pre-trained GAN (*e.g.*, StyleGAN2) to restore realistic faces while precerving fidelity. <br>

If you want to use the paper model, please go to this [Colab Demo](https://colab.research.google.com/drive/1Oa1WwKB4M4l1GmR7CtswDVgOCOeSLChA?usp=sharing) for GFPGAN <a href="https://colab.research.google.com/drive/1Oa1WwKB4M4l1GmR7CtswDVgOCOeSLChA?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="google colab logo"></a>.

**Limitations**: GFPGAN could not handle all the low-quality faces in the real world. Therefore, it may fail on your own cases.

###Enjoy! :-)

<img src="https://xinntao.github.io/projects/GFPGAN_src/gfpgan_teaser.jpg" width="800">


# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.


### Mount GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Git clone repo

In [ ]:
# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.3 model for the demo
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
!apt install detox
!pip install Pillow

import shutil
import os
upload_folder = 'inputs/upload'

# 2. Upload Images / Use the demo images

## OPTION: Clean and move files to upload folder

In [ ]:
!rm /content/GFPGAN/inputs/upload/*

In [ ]:
!mkdir /content/GFPGAN/inputs/upload

In [ ]:
!cp -v /content/drive/MyDrive/AI/stable_diffusion/to_upscale/asian_female_polaroid_aging/* /content/GFPGAN/inputs/upload/

## sanitize input filenames

In [ ]:
!detox -r /content/GFPGAN/inputs/upload/

# Downsample images

In [ ]:
import glob, os
import os, sys
from PIL import Image


size = 256, 256
# size = 128, 128

os.chdir("/content/GFPGAN/inputs/upload")
for infile in glob.glob("*.png"):
    print(infile)

    outfile = os.path.splitext(infile)[0] + ".jpg"
    if infile != outfile:
        try:
            im = Image.open(infile)
            im.thumbnail(size, Image.ANTIALIAS)
            im.save(outfile, "JPEG")
        except IOError:
            print("cannot create thumbnail for '%s'" % infile)

In [ ]:
!rm /content/GFPGAN/inputs/upload/*.png

## 3. Inference

In [ ]:
%cd /content/GFPGAN

In [ ]:
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
# You can find the different models in https://github.com/TencentARC/GFPGAN#european_castle-model-zoo
!rm -rf /content/GFPGAN/results/*
!mkdir /content/GFPGAN/results
!python /content/GFPGAN/inference_gfpgan.py -i /content/GFPGAN/inputs/upload -o /content/GFPGAN/results -v 1.3 -s 4 --bg_upsampler realesrgan

# Usage: python inference_gfpgan.py -i inputs/whole_imgs -o results -v 1.3 -s 2 [options]...
# 
#  -h                   show this help
#  -i input             Input image or folder. Default: inputs/whole_imgs
#  -o output            Output folder. Default: results
#  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3. Default: 1.3
#  -s upscale           The final upsampling scale of the image. Default: 2
#  -bg_upsampler        background upsampler. Default: realesrgan
#  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
#  -suffix              Suffix of the restored faces
#  -only_center_face    Only restore the center face
#  -aligned             Input are aligned faces
#  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto

!ls /content/GFPGAN/results/

## 4. Visualize -- deactivated for faster execution

In [ ]:
# # We first visualize the cropped faces
# # The left are the inputs images; the right are the results of GFPGAN

# import cv2
# import matplotlib.pyplot as plt
# def display(img1, img2):
#   fig = plt.figure(figsize=(25, 10))
#   ax1 = fig.add_subplot(1, 2, 1) 
#   plt.title('Input image', fontsize=16)
#   ax1.axis('off')
#   ax2 = fig.add_subplot(1, 2, 2)
#   plt.title('GFPGAN output', fontsize=16)
#   ax2.axis('off')
#   ax1.imshow(img1)
#   ax2.imshow(img2)
# def imread(img_path):
#   img = cv2.imread(img_path)
#   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#   return img

# # display each image in the upload folder
# import os
# import glob

# input_folder = 'results/cropped_faces'
# result_folder = 'results/restored_faces'
# input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
# output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
# for input_path, output_path in zip(input_list, output_list):
#   img_input = imread(input_path)
#   img_output = imread(output_path)
#   display(img_input, img_output)


In [ ]:
# # We then visualize the whole image
# # The left are the inputs images; the right are the results of GFPGAN

# import cv2
# import matplotlib.pyplot as plt
# def display(img1, img2):
#   fig = plt.figure(figsize=(25, 10))
#   ax1 = fig.add_subplot(1, 2, 1) 
#   plt.title('Input image', fontsize=16)
#   ax1.axis('off')
#   ax2 = fig.add_subplot(1, 2, 2)
#   plt.title('GFPGAN output', fontsize=16)
#   ax2.axis('off')
#   ax1.imshow(img1)
#   ax2.imshow(img2)
# def imread(img_path):
#   img = cv2.imread(img_path)
#   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#   return img

# # display each image in the upload folder
# import os
# import glob

# input_folder = 'inputs/upload'
# result_folder = 'results/restored_imgs'
# input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
# output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
# for input_path, output_path in zip(input_list, output_list):
#   img_input = imread(input_path)
#   img_output = imread(output_path)
#   display(img_input, img_output)

## 5. Archive results

## option faces only

In [ ]:
!rm /content/drive/MyDrive/AI/face_restore/output/*

In [ ]:
!cp /content/GFPGAN/results/restored_faces/* /content/drive/MyDrive/AI/face_restore/output

In [ ]:
!mkdir /content/drive/MyDrive/AI/face_restore/$(date +%Y%m%d_%H%M%S) && mv /content/drive/MyDrive/AI/face_restore/output/*.png /content/drive/MyDrive/AI/face_restore/$(date +%Y%m%d_%H%M%S)

# option full image also

In [ ]:
!rm /content/drive/MyDrive/AI/face_restore/output/*

In [ ]:
!cp /content/GFPGAN/results/restored_imgs/* /content/drive/MyDrive/AI/face_restore/output

In [ ]:
!mkdir /content/drive/MyDrive/AI/face_restore/$(date +%Y%m%d_%H%M%S) && mv /content/drive/MyDrive/AI/face_restore/output/*.jpg /content/drive/MyDrive/AI/face_restore/$(date +%Y%m%d_%H%M%S)